<a href="https://colab.research.google.com/github/jmcaussade/Arreglo-Incongruencias-Juegos-Jardin/blob/main/ArregloAtributosJuegosMesaJardin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>IMPORTATIONS</h1>

In [13]:
import os

<h1>Connection to Google Drive</h1>


In [14]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
print("Hello world")

Hello world


In [37]:
print("Message 2")

Message 2
